## Лабораторная работа 12

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import dask.bag as db
import json
import re
import pandas as pd

In [2]:
import dask

In [3]:
dask.__version__

'2022.7.0'

* [reviews_full.zip](https://drive.google.com/file/d/1iSIF9qeZqUoiLawJ0w6KpH0lV-31Lnf0/view?usp=share_link )

1. В файлах архиве `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива в виде `dask.bag`. Преобразуйте текстовое содержимое файлов в объекты python (с помощью модуля `json`). Выведите на экран первые 5 элементов полученного `bag`.

In [5]:
b = db.read_text(urlpath="./reviews_full/reviews_full/reviews_*.json").map(json.loads)
b.take(5)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.'},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten'},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will be making this often.'},
 {'u

2. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path`), использовав для этого регулярное выражение.

In [6]:
import re
from datetime import datetime

In [8]:
def my_loader(data) -> dict:

    json_str, path = data
    file_number = re.findall('reviews_([0-9]).json', path)    
    json_obj = json.loads(json_str)
    json_obj["rating"] = int(file_number[0])
    json_obj["date"] = datetime.strptime(json_obj["date"], '%Y-%m-%d')

    return json_obj

b_bag = db.read_text(urlpath="./reviews_full/reviews_full/reviews_*.json", include_path=True).map(my_loader)
b_bag.take(50)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': datetime.datetime(2016, 5, 8, 0, 0),
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.',
  'rating': 0},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': datetime.datetime(2006, 6, 14, 0, 0),
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten',
  'rating': 0},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': datetime.datetime(1985, 11, 19, 0, 0),
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a pe

3. Посчитайте количество отзывов в исходном датасете.

In [9]:
b_bag.count().compute()

9057540

4. Отфильтруйте `bag`, сохранив только отзывы, оставленные в 2014 и 2015 годах.

In [10]:
def years_filter(item):
    return item["date"].year in (2014, 2015)

In [11]:
c_bag = b_bag.filter(years_filter)
c_bag.take(5)

({'user_id': 229850,
  'recipe_id': 1300038,
  'date': datetime.datetime(2014, 10, 3, 0, 0),
  'review': 'Took this to a New Year&#039;s Eve Party. Everyone loved it! It&#039;s absolutely perfect, the flavor, the crunch, just delicious!',
  'rating': 0},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': datetime.datetime(2015, 5, 8, 0, 0),
  'review': 'Simple and easy way to enjoy a slice of pizza any time!  Well-toasted bread is the key - really toast it!  I put a bit of pizza sauce underneath my cheese for a more pizza-like flavor.  I used sourdough bread & medium cheddar cheese.  Fast & fun!  Great idea!  Made for 1-2-3 Hits Tag Game.',
  'rating': 0},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': datetime.datetime(2015, 6, 30, 0, 0),
  'review': 'Delish!  I wanted to make this spicy so I used hot enchilada sauce and jalapeno refried beans.  I forgot to buy the onions so I doctored up the beans with onion powder and granulated garlic.  Added the olives under the cheese 

5. Выполните препроцессинг отзывов:
    * привести строки к нижнему регистру
    * обрезать пробельные символы в начале и конце строки
    * удалите все символы, кроме английских букв и пробелов
    
Примените препроцессинг ко всем записям из `bag`, полученного в задании 4.

In [12]:
import re
pattern = re.compile('[^a-z ]+')

In [13]:
def my_preprocessing(item):
    current_str = item["review"].lower().strip()
    item["review"] = re.sub(pattern, '', current_str)
    return item

6. Посчитайте количество отзывов в датасете, полученном в результате решения задачи 5. В случае ошибок прокомментируйте результат и исправьте функцию препроцессинга.

In [15]:
c_len = c_bag.count().compute()
print(c_len)

735274


7. Посчитайте, как часто в наборе, полученном в задании 5, встречается та или иная оценка

In [17]:
for item in c_bag.pluck(key="rating").frequencies().compute():
    print("Rating={}, count={}".format(*item))

Rating=0, count=42472
Rating=1, count=9246
Rating=2, count=9380
Rating=3, count=26532
Rating=4, count=119413
Rating=5, count=528231


8. Найдите среднее значение `rating` в выборке

In [18]:
c_bag.pluck(key="rating").mean().compute()

4.388036296673077

9. Используя метод `foldby`, подсчитать максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 5.

In [19]:
def combine(left, right):
    """Возврат максимального значения из всех сегментов"""
    return max(left, right)

def binop(previous, current):
    """Подсчет максимума внутри определенного сегмента"""
    return max(previous, len(current["review"]))

result = c_bag.foldby(key="rating", binop=binop, initial=0, combine=combine, combine_initial=0).compute()
for item in result:
    print("Rating={}, max review len={}".format(*item))

Rating=0, max review len=6972
Rating=1, max review len=3347
Rating=2, max review len=3040
Rating=3, max review len=3308
Rating=4, max review len=6972
Rating=5, max review len=5799


In [ ]:
import matplotlib.pyplot as plt
import math

def ebbinghaus_forgetting_curve(initial_memorization, days):
    retention_rate = [1.0, 0.5, 0.3, 0.2, 0.15, 0.1, 0.05, 0.02]
    memory_strengths = [initial_memorization]

    for day in range(1, days+1):
        strength = memory_strengths[-1]
        if day <= 8:
            strength *= retention_rate[day-1]
        else:
            strength *= math.exp(-0.1*(day-8))
        memory_strengths.append(strength)
        
    return memory_strengths

# Example usage
initial_memorization = 100
days = 30
memory_strengths = ebbinghaus_forgetting_curve(initial_memorization, days)
plt.plot(range(days+1), memory_strengths)
plt.xlabel('Days')
plt.ylabel('Memory Strength')
plt.title('Ebbinghaus Forgetting Curve')
plt.show()
